# Sentiment Analysis with BERT

In [17]:
# pip install datasets // datasets from hugging face transformers
from datasets import load_dataset

In [18]:
raw_datasets = load_dataset("imdb")

Reusing dataset imdb (C:\Users\HP\.cache\huggingface\datasets\imdb\plain_text\1.0.0\2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


  0%|          | 0/3 [00:00<?, ?it/s]

In [19]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [20]:
raw_datasets['train']['label'][0]

0

In [21]:
raw_datasets['train']['text'][0]

'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, ev

In [22]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

In [23]:
def tokenize(sequence):
    tokens = tokenizer.encode_plus(sequence, max_length=512,
                                   truncation=True, padding='max_length',
                                   add_special_tokens=True, return_token_type_ids=False,
                                   return_tensors='tf')
    return tokens['input_ids'], tokens['attention_mask']

In [24]:
import numpy as np
num_of_elements = 5000

Xids = np.zeros((num_of_elements, 512))
Xmask = np.zeros((num_of_elements, 512))

idx = np.random.randint(0, 25000, num_of_elements)  
small_dataset = np.array(raw_datasets['train']['text'])[idx.astype(int)]
small_dataset_labels = np.array(raw_datasets['train']['label'])[idx.astype(int)]

labels = np.array(small_dataset_labels)

In [25]:
for i, sequence in enumerate(small_dataset):
    tokens = tokenize(sequence)
    Xids[i, :], Xmask[i, :] = tokens[0], tokens[1]

In [26]:
 Xids[0]

array([  101., 12786.,  2339.,   117., 10928.,   188.,  6617.,   118.,
       20497., 11826.,   117.,  1189.,  1114.,  1126.,  1593.,  1703.,
        2960.,  1104., 10774.,   113.,   170.,  1654.,  1395.,  1554.,
        1104.,  7565.,  1105.,  1168.,  5197.,  1215.,  1106.,  3531.,
        1105.,  1260.,  6617., 15940.,  7416.,  1121.,  6144.,  2000.,
        1110.,  1576.,  1118.,  1178., 24497.,  9960.,  2249.,   117.,
        1105.,  1110.,   159.,  9637.,  3663.,  9874., 14385.,  1120.,
        1480.,   114.,   117.,  1105.,  1554.,  1104.,  3227.,  1183.,
        1839.,  3154.,   119.,  4978.,  2499.,  1110.,  1136.,  1178.,
       15445.,   117.,  1133.,  1119.,  1145.,  3114.,  1141.,  1104.,
        1117.,  1374.,   107.,   146.,   112.,   182.,  1303., 10802.,
        1111.,  1103.,  1948.,   107.,  3853.,   119.,   113.,   115.,
         122.,   120.,   123.,   114.,   102.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
      

In [27]:
from transformers import TFAutoModel

In [28]:
bert = TFAutoModel.from_pretrained('bert-base-cased')

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [29]:
bert.summary()

Model: "tf_bert_model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  108310272 
Total params: 108,310,272
Trainable params: 108,310,272
Non-trainable params: 0
_________________________________________________________________


In [30]:
import tensorflow as tf

input_ids = tf.keras.layers.Input(shape=(512,), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(512,), name='attention_mask', dtype='int32')

embeddings = bert.bert(input_ids, attention_mask=mask)[0]  # we access the transformer model within our bert object using the bert attribute (eg bert.bert instead of bert)

x = tf.keras.layers.Dropout(0.1)(embeddings)
x = tf.keras.layers.GlobalMaxPool1D()(x)
x = tf.keras.layers.Dense(128, activation ='relu')(x)
y = tf.keras.layers.Dense(1, activation ='sigmoid', name='outputs')(x)

model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

model.layers[2].trainable = False

In [31]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 512)]        0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 512)]        0                                            
__________________________________________________________________________________________________
bert (TFBertMainLayer)          TFBaseModelOutputWit 108310272   input_ids[0][0]                  
                                                                 attention_mask[0][0]             
__________________________________________________________________________________________________
dropout_75 (Dropout)            (None, 512, 768)     0           bert[0][0]                 

In [32]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=["accuracy"])

In [33]:
history = model.fit(
    [Xids, Xmask], labels,
    validation_split=0.8,
    batch_size = 128,
    verbose = 1,
    epochs=1)

8/8 [==============================] - 3660s 505s/step - loss: 1.5588 - accuracy: 0.4925 - val_loss: 0.7086 - val_accuracy: 0.4939
